<a href="https://colab.research.google.com/github/pingao2019/DS-Unit-2-Linear-Models/blob/master/PingCopy_of_h3Copy_of_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ x] Do one-hot encoding of categorical features.
- [ x] Do feature selection with `SelectKBest`.
- [ x] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ x] Get mean absolute error for the test set.
- [ x] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
import numpy as np
import pandas as pd




In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
#df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].astype(float, copy=False)

In [0]:
#df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].astype(float)

In [0]:
print(df.shape)
df.head()


(3164, 19)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


 Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.


In [0]:
df.describe(include= 'number')

,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.0,3.164000e+03
mean,6908.597029,75.847029,11027.442162,0.987358,0.015803,1.003161,1469.718394,1943.639697,1.0,6.281899e+05
std,3964.333893,160.779187,482.591574,0.114537,0.127241,0.172362,586.645088,26.679176,0.0,2.969134e+05
min,21.000000,1.000000,10030.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,1.000000e+05
25%,4003.250000,21.000000,10461.000000,1.000000,0.000000,1.000000,1144.000000,1925.000000,1.0,4.450000e+05
50%,6269.500000,42.000000,11235.000000,1.000000,0.000000,1.000000,1360.000000,1938.000000,1.0,5.650000e+05
75%,10206.250000,69.000000,11413.000000,1.000000,0.000000,1.000000,1683.000000,1955.000000,1.0,7.600000e+05
max,16350.000000,2720.000000,11697.000000,2.000000,2.000000,3.000000,7875.000000,2018.000000,1.0,2.000000e+06


In [0]:
condition= (df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS' )
df=df[condition]

df= df[(df['SALE_PRICE'] >=1.0e+05)&(df['SALE_PRICE'] <=2.0e+06)]
print(df.shape)
df.head()


(3164, 19)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [0]:
#df= df.drop(['EASE-MENT', 'APARTMENT_NUMBER'], axis=1)

In [0]:
df.dropna(inplace=True)

In [0]:
df.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [0]:
 df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].apply(lambda x: 1 if x==0 else 0)


Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
Do one-hot encoding of categorical features

In [0]:
import datetime
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
condtion = (df['SALE_DATE']==2019)
df['year'] = pd.DatetimeIndex(df['SALE_DATE']).year




In [0]:
df['month'] = pd.DatetimeIndex(df['SALE_DATE']).month
df.head(2)



,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,year,month
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,0,1325.0,1930.0,1,A9,550000,2019-01-01,2019,1
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,0,2001.0,1940.0,1,A1,200000,2019-01-01,2019,1


Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test. Do one-hot encoding of categorical features

In [0]:
condtion= (df['month']==1) 
df1=df[condtion]
condtion= (df['month']==2) 
df2=df[condtion]
condtion= (df['month']==3) 
df3=df[condtion]
train = pd.concat([df1, df2, df3])
print(train.shape)
train.tail()


(2517, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,year,month
18129,5,OTHER,01 ONE FAMILY DWELLINGS,1,4081,44,A2,10 SEAFOAM STREET,10306.0,1.0,0.0,1.0,0,921.0,1950.0,1,A2,330000,2019-03-29,2019,3
18130,5,OTHER,01 ONE FAMILY DWELLINGS,1,2373,201,A5,74 MCVEIGH AVE,10314.0,1.0,0.0,1.0,0,2128.0,1980.0,1,A5,690000,2019-03-29,2019,3
18132,5,OTHER,01 ONE FAMILY DWELLINGS,1,1132,42,A1,479 VILLA AVENUE,10302.0,1.0,0.0,1.0,0,1807.0,2018.0,1,A1,610949,2019-03-29,2019,3
18134,5,OTHER,01 ONE FAMILY DWELLINGS,1,3395,37,A2,63 NUGENT AVENUE,10305.0,1.0,0.0,1.0,0,621.0,1930.0,1,A2,520000,2019-03-29,2019,3
18147,4,OTHER,01 ONE FAMILY DWELLINGS,1,11159,11,A0,223-29 103RD AVENUE,11429.0,1.0,0.0,1.0,0,1163.0,1950.0,1,A0,104000,2019-03-30,2019,3


In [0]:
condtion= (df['month']==4) 
test=df[condtion]
print(test.shape)
test.head()


(647, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,year,month
18235,2,OTHER,01 ONE FAMILY DWELLINGS,1,5913,878,A1,4616 INDEPENDENCE AVENUE,10471.0,1.0,0.0,1.0,0,2272.0,1930.0,1,A1,895000,2019-04-01,2019,4
18239,2,OTHER,01 ONE FAMILY DWELLINGS,1,5488,48,A2,558 ELLSWORTH AVENUE,10465.0,1.0,0.0,1.0,0,720.0,1935.0,1,A2,253500,2019-04-01,2019,4
18244,3,OTHER,01 ONE FAMILY DWELLINGS,1,5936,31,A1,16 BAY RIDGE PARKWAY,11209.0,1.0,0.0,1.0,0,2210.0,1925.0,1,A1,1300000,2019-04-01,2019,4
18280,3,OTHER,01 ONE FAMILY DWELLINGS,1,7813,24,A5,1247 EAST 40TH STREET,11210.0,1.0,0.0,1.0,0,1520.0,1915.0,1,A5,789000,2019-04-01,2019,4
18285,3,OTHER,01 ONE FAMILY DWELLINGS,1,8831,160,A9,2314 PLUMB 2ND STREET,11229.0,1.0,0.0,1.0,0,840.0,1925.0,1,A9,525000,2019-04-01,2019,4


Do one-hot encoding of categorical features.
 Do feature selection with SelectKBest.
 Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)
 Get mean absolute error for the test set.

In [0]:
train.describe(exclude= 'number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2517,2517,2517,2517,2517,2517,2517,2517
unique,5,7,1,2,13,2507,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,118-20 202ND STREET,A1,2019-01-31 00:00:00
freq,1209,2368,2517,2486,921,2,921,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [0]:
train['BOROUGH'].value_counts()



4    1209
5     664
3     401
2     242
1       1
Name: BOROUGH, dtype: int64

Now look at the relationship between  SALE PRICE and some feature:

In [0]:
train.columns.to_list()

['BOROUGH',
 'NEIGHBORHOOD',
 'BUILDING_CLASS_CATEGORY',
 'TAX_CLASS_AT_PRESENT',
 'BLOCK',
 'LOT',
 'BUILDING_CLASS_AT_PRESENT',
 'ADDRESS',
 'ZIP_CODE',
 'RESIDENTIAL_UNITS',
 'COMMERCIAL_UNITS',
 'TOTAL_UNITS',
 'LAND_SQUARE_FEET',
 'GROSS_SQUARE_FEET',
 'YEAR_BUILT',
 'TAX_CLASS_AT_TIME_OF_SALE',
 'BUILDING_CLASS_AT_TIME_OF_SALE',
 'SALE_PRICE',
 'SALE_DATE',
 'year',
 'month']

In [0]:
target = 'SALE_PRICE'
high_cardinality = [

 'BLOCK',
 'LOT',
 
 'BUILDING_CLASS_AT_PRESENT',
 'ADDRESS',
 
 'ZIP_CODE',
 'RESIDENTIAL_UNITS',
 'COMMERCIAL_UNITS',
  
 'TAX_CLASS_AT_TIME_OF_SALE',
 'BUILDING_CLASS_AT_TIME_OF_SALE',
 
 'SALE_DATE',
 'YEAR_BUILT',
 'year',
 'month']
features = train.columns.drop([target] + high_cardinality)

In [0]:
train.groupby('BOROUGH')['SALE_PRICE'].mean().sort_values()



BOROUGH
2    473104.623967
5    535988.615964
4    646734.679901
1    739000.000000
3    773207.940150
Name: SALE_PRICE, dtype: float64

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

Use the fit_transform method on the train set.   
Use the transform method on validation / test sets

In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)


In [0]:
X_test = encoder.transform(X_test)

In [0]:
print(X_train.shape)

X_train


(2517, 18)


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET
44,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1325.0
61,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,2001.0
78,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,2043.0
108,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,2680.0
111,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1872.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,1.0,0,921.0
18130,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,1.0,0,2128.0
18132,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1807.0
18134,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,1.0,0,621.0


In [0]:
print(X_test.shape)
X_test

(647, 18)


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET
18235,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,2272.0
18239,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,720.0
18244,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,2210.0
18280,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1520.0
18285,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,840.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1478.0
23031,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1280.0
23032,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1333.0
23033,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1.0,0,1020.0


In [0]:
test.isnull().sum()

In [0]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=13)


# .fit_transform on the train set
# .transform on test set

X_train_selected = selector.fit_transform(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_e

In [0]:
X_test_selected = selector.transform(X_test)

In [0]:
X_train_selected.shape, X_test_selected.shape

((2517, 13), (647, 13))

In [0]:
# features were selected
selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
TOTAL_UNITS
GROSS_SQUARE_FEET


Features not selected:
BOROUGH_1
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
LAND_SQUARE_FEET


In [0]:
#Ridge Regression
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge

from sklearn.metrics import r2_score, mean_squared_error


In [0]:
for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $185,788 

2 features
Test Mean Absolute Error: $181,871 

3 features
Test Mean Absolute Error: $173,292 

4 features
Test Mean Absolute Error: $172,669 

5 features
Test Mean Absolute Error: $171,216 

6 features
Test Mean Absolute Error: $170,983 

7 features
Test Mean Absolute Error: $169,889 

8 features
Test Mean Absolute Error: $169,779 

9 features
Test Mean Absolute Error: $170,305 

10 features
Test Mean Absolute Error: $169,357 

11 features
Test Mean Absolute Error: $170,084 

12 features
Test Mean Absolute Error: $170,084 

13 features
Test Mean Absolute Error: $171,412 

14 features
Test Mean Absolute Error: $171,414 

15 features
Test Mean Absolute Error: $171,414 

16 features
Test Mean Absolute Error: $171,414 

17 features
Test Mean Absolute Error: $171,414 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

### we can  choose 8 features (Test Mean Absolute Error: $169,779) as best fit.

Fit a ridge regression model with multiple features. 

Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)
Get mean absolute error for the test set.

In [0]:

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#  Ridge
from sklearn.linear_model import LinearRegression, Ridge
# model evaluation
from sklearn.metrics import r2_score, mean_squared_error

In [0]:
# Using CV
    from sklearn.linear_model import Ridge
    alphas = [0.1, 1, 10, 100, 1e3, 1e4, 2e4, 5e4, 8e4, 1e5, 1e6, 1e7, 1e8]
    reg = RidgeCV(alphas=alphas, store_cv_values=True)
    #reg.fit(X_train_selected, y_train, sample_weight=sample_weight)
    reg.fit(X_train_selected, y_train)
    cv_mse = np.mean(reg.cv_values_, axis=0)
    print("alphas: %s" % alphas)
    print("CV MSE: %s" % cv_mse)
    print("Best alpha using built-in RidgeCV: %f" % reg.alpha_)

    # generate the prediction using the best model
    alpha = reg.alpha_
    reg = Ridge(alpha=alpha)
    #reg.fit(X_train_selected, y_train, sample_weight=sample_weight)
    reg.fit(X_train_selected, y_train)
    predict_y = reg.predict(X_test_selected)
    train_y_pred = reg.predict(X_train_selected)

    #return {"y": predict_y, "train_y": train_y_pred, "coef": reg.coef_} 

alphas: [0.1, 1, 10, 100, 1000.0, 10000.0, 20000.0, 50000.0, 80000.0, 100000.0, 1000000.0, 10000000.0, 100000000.0]
CV MSE: [5.64737657e+10 5.65476509e+10 5.77863407e+10 5.96102365e+10
 6.43372837e+10 6.85101786e+10 6.89235899e+10 6.91889111e+10
 6.92574232e+10 6.92804585e+10 6.93639091e+10 6.93707852e+10
 6.95182434e+10]
Best alpha using built-in RidgeCV: 0.100000


### Conclusion: We got best alpha=0.1 by using  RidgeCV.